# This notebook contains an analysis of the efficacy of various decontamination procedures #

## Figure 2 ans Supplementary Figure 1c can be generated using this notebook##

### Import necessary modules ###

In [1]:
import pandas as pd
from scipy.stats import linregress, ttest_ind
import numpy as np
import sys
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"

if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML
    HTML("""
         <script>
          var waitForPlotly = setInterval( function() {
          if( typeof(window.Plotly) !== "undefined" ){
          MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
          MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
          clearInterval(waitForPlotly);}}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)

### Load contamination rate data from several experiments ###

In [2]:
contam_data = pd.read_excel("Consolidated_Contamination_Data.xlsx")
contam_data['sterility'] = 100 - contam_data['contamination_rate']

### Plot initial test data - Decontamination with 0 hold time and 4 washes with default (10$\mu$L) air gap (Fig. 2B) ###

In [3]:
colors = ['#927fbf','#a8d5ff','#2797ff','#0062bd']
vary_disinfectant_0STAG = contam_data[contam_data['stag']==10]
disinfectants = ['12% Sodium<br>hypochlorite',  '6% Sodium<br>hypochlorite',
                 '3% Sodium<br>hypochlorite', 'Water']

trace_list = []

trace_list.append(go.Bar(y=list(reversed(disinfectants)), x=list(reversed(vary_disinfectant_0STAG['sterility'])),
                         marker=dict(color=colors, line=dict(width=0, color='black')),
                         name='Contamination Rate<br>(% contaminated wells)', showlegend=False,
                         orientation='h'))

trace_list.append(go.Scatter(x=[100]*10, y=np.linspace(0,10,10),
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             yaxis='y2',showlegend=False, visible=True))

layout = go.Layout(height=350, width=350, showlegend=True, bargap=0.5, 
                   legend_orientation='h',

                    yaxis=dict(title_text='Disinfectant', 
                               title_font=dict(family='Arial',size=16,color='black'), title_standoff=0, automargin=True,
                               showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                               ticks='outside', ticklen=4, tickfont=dict(size=14, family='Arial', color='black'),
                               tickangle=-0, tickcolor='black', tickvals=disinfectants),
                   yaxis2=dict(visible=False, anchor='x', overlaying='y', range=(0,10)),

                   xaxis=dict(title='Sterility<br>(% sterile wells)',
                              titlefont=dict(family='Arial', size=16, color='black'), title_standoff=0,
                              showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                              ticks='outside', ticklen=4, tickfont=dict(family='Arial',size=14, color='black'),
                              nticks=10, tickangle=0, tickcolor='black',range=(0,105)))

fig = go.Figure(data=trace_list,layout=layout)
plot(fig)
#pio.write_image(fig,"Figures/fig1.svg",format='svg')


### Plot air gap variation data - Decontamination with different hold times, washes and air gaps (Fig. 2D)

In [4]:
stag0data = contam_data[(contam_data['stag']==0)&(contam_data['wash_number']!=0)]
stag50data = contam_data[(contam_data['stag']==50)&(contam_data['wash_number']!=0)]
stag100data = contam_data[(contam_data['stag']==100)&(contam_data['wash_number']!=0)]
stag250data = contam_data[(contam_data['stag']==250)&(contam_data['wash_number']!=0)][6:]
stag_data = pd.concat([stag0data, stag50data, stag100data, stag250data]).sort_values(by=['stag','hold_time','wash_number'])

In [5]:
colors = [['#ffb6ad', '#ff7362', '#c13223'],
          ['#ffdd91', '#f5be46', '#eda911'],
          ['#a8d5ff', '#2797ff', '#0062bd'],
          ['#b3e2b5', '#73ca76', '#37923b']]

colors = ['#b3e2b5', '#73ca76', '#37923b']

for j,air_gap in enumerate(stag_data['stag'].unique()):

    trace_list = []
    trace_list.append(go.Scatter(y=[100]*10, x=np.linspace(0,10,10),
                             name='Required Sterility<br>(100%)',
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             xaxis='x2',showlegend=True, visible=True))
    for i,hold_time in enumerate(list(stag_data['hold_time'].unique())):

        trace_list.append(go.Bar(x=stag_data[(stag_data['stag']==air_gap) & (stag_data['hold_time']==hold_time)].wash_number.to_list(),
                                 y=stag_data[(stag_data['stag']==air_gap) & (stag_data['hold_time']==hold_time)].sterility.to_list(),
                                 marker=dict(color=colors[i], line=dict(width=0, color='black')),
                                 name='Hold Time:'+str(hold_time), showlegend=True))

    layout = go.Layout(height=400, width=475, showlegend=True, bargroupgap=0.1, bargap=0.3,
                       legend_orientation='v',legend_y=-0, legend_font=dict(family='Arial', color='black', size=14),
                       xaxis2=dict(visible=False, anchor='y', overlaying='x', range=(0,4)),

                        xaxis=dict(title_text='Number of Washes', 
                                   title_font=dict(family='Arial',size=16,color='black'), title_standoff=0,
                                   showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                                   ticks='outside', ticklen=4, tickfont=dict(size=16, family='Arial', color='black'),
                                   tickangle=-0, tickcolor='black'),

                       yaxis=dict(title='Sterility<br>(% sterile wells)',
                                  titlefont=dict(family='Arial', size=16, color='black'), title_standoff=0,
                                  showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                                  ticks='outside', ticklen=4, tickfont=dict(family='Arial',size=16, color='black'),
                                  tickangle=0, nticks=10, tickcolor='black',range=(0,105)))

    fig = go.Figure(data=trace_list,layout=layout)
    plot(fig)
    #pio.write_image(fig,"Figures/fig1_d_"+str(air_gap)+".svg",format='svg')


### Plot air gap variation data - Decontamination with different air gaps (Supplementary Fig. S1C)

In [6]:
colors = ['#ffb6ad','#ffdd91','#a8d5ff','#b3e2b5']
colors = ['#ff7362', '#f5be46', '#2797ff', '#73ca76']
colors = ['#a8d5ff','#71baff', '#2797ff','#0062bd']

stag_group=stag_data.groupby('stag').agg('sum')[['total_wells','wells_contaminated']]
disinfectants = ['12% Sodium<br>hypochlorite',  '6% Sodium<br>hypochlorite',
                 '3% Sodium<br>hypochlorite', 'Water']

trace_list = []
trace_list.append(go.Scatter(y=[100]*10, x=np.linspace(0,10,10),
                         name='Required Sterility<br>(100%)',
                         mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                         xaxis='x2',showlegend=True, visible=True))
trace_list.append(go.Bar(x=stag_group.index,
                         y=(stag_group.total_wells - stag_group.wells_contaminated)/stag_group.total_wells*100,
                         marker=dict(color=colors, line=dict(width=0, color='black')),
                         name='Contamination Rate<br>(% contaminated wells)', showlegend=False,
                         orientation='v'))

layout = go.Layout(height=400, width=475, showlegend=True, bargap=0.6, 
                   legend_orientation='v', legend_font=dict(family='Arial', color='black', size=14),
                    xaxis2=dict(visible=False, anchor='y', overlaying='x', range=(0,4)),

                    xaxis=dict(title_text='AirGap (\u03BCL)', 
                               title_font=dict(family='Arial',size=16,color='black'), title_standoff=0, automargin=True,
                               showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                               ticks='outside', ticklen=4, tickfont=dict(size=16, family='Arial', color='black'),
                               tickangle=-0, tickcolor='black',type='category'),

                   yaxis=dict(title='Sterility<br>(% sterile wells)',
                              titlefont=dict(family='Arial', size=16, color='black'), title_standoff=0,
                              showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                              ticks='outside', ticklen=4, tickfont=dict(family='Arial',size=16, color='black'),
                              nticks=10, tickangle=0, tickcolor='black',range=(0,105)))

fig = go.Figure(data=trace_list,layout=layout)
plot(fig)
#pio.write_image(fig,"Figures/fig_airgapeffect.svg",format='svg')



### Final decontamination test with 250 $\mu$L air gap and 2 washes, 0 hold time. Figure 2E.###

In [7]:
vary_disinf_stag250 = contam_data[contam_data['stag']==250][:6]

colors = list(reversed(['#0062bd','#2797ff','#71baff', '#a8d5ff','#0062bd','#a8d5ff']))

disinfectants = ['12% Sodium<br>hypochlorite',  '9% Sodium<br>hypochlorite', '6% Sodium<br>hypochlorite',
                 '3% Sodium<br>hypochlorite', '70% Ethanol', 'Water']

trace_list = []
trace_list.append(go.Scatter(x=[100]*10, y=np.linspace(0,10,10),
                             mode='lines', line=dict(dash='dot',color='#9b5151',width=1),
                             yaxis='y2',showlegend=False, visible=True))

trace_list.append(go.Bar(y=list(reversed(disinfectants)), x=list(reversed(list(vary_disinf_stag250['sterility']))),
                         marker=dict(color=colors, line=dict(width=0, color='black')),
                         name='Contamination Rate<br>(% contaminated wells)', showlegend=False,
                         orientation='h'))

layout = go.Layout(height=400, width=375, showlegend=True, bargap=0.5, 
                   legend_orientation='h',
                   yaxis2=dict(visible=False, anchor='x', overlaying='y', range=(0,10)),

                    yaxis=dict(title_text='Disinfectant', 
                               title_font=dict(family='Arial',size=16,color='black'), title_standoff=0, automargin=True,
                               showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                               ticks='outside', ticklen=4, tickfont=dict(size=12, family='Arial', color='black'),
                               tickangle=-0, tickcolor='black', tickvals=disinfectants),

                   xaxis=dict(title='Sterility<br>(% sterile wells)',
                              titlefont=dict(family='Arial', size=16, color='black'), title_standoff=0,
                              showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,
                              ticks='outside', ticklen=4, tickfont=dict(family='Arial',size=14, color='black'),
                              nticks=10, tickangle=0, tickcolor='black',range=(0,105)))

fig = go.Figure(data=trace_list,layout=layout)
plot(fig)


#pio.write_image(fig,"Figures/fig1_last.svg",format='svg')
